In [7]:

# stacked generalization with neural net meta model on blobs dataset
# from sklearn.datasets import make_blobs
# from sklearn.metrics import accuracy_score
# from keras.models import load_model
# from keras.utils import to_categorical
# from keras.utils import plot_model
# from keras.models import Model
# from keras.layers import Input
# from keras.layers import Dense
# from keras.layers import concatenate
# from numpy import argmax
# from numpy import dstack
# import os
# from keras.utils import to_categorical
# from skimage.transform import resize
# import os
# import numpy as np
# from keras.models import load_model, Model, Sequential
# from keras.layers import Concatenate, Dense
# from keras.optimizers import Adam
# stacked generalization with linear meta model on blobs dataset
# from sklearn.datasets import make_blobs
# from keras.utils import to_categorical
# from numpy import dstack
import os
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
# from keras.applications import vgg16
# from keras.applications import xception
from keras.applications import mobilenet_v2, vgg16, vgg19, resnet_v2, xception
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
%matplotlib inline


In [8]:
# load models from file
def load_all_models(models_dir):
    # model_paths = ['model1.h5', 'model2.h5', 'model3.h5']
    models_path = [model for model in os.listdir(models_dir)]
    models = []
    for path in models_path:
        model_path = f"{models_dir}\\{path}"
        model = load_model(model_path)
        model.trainable = False
        models.append(model)

    # for model in models:
    #     model.summary()

    for i in range(len(models)):
        model = models[i]
        for layer in model.layers:
            layer._name = f"ensemble_{i}_{layer.name}"

    return models


# load all models
members = load_all_models(f"{os.getcwd()}\\models\\")
print("Loaded %d models" % len(members))


Loaded 3 models


In [9]:
# Loading the data and generating labels
IMG_SIZE = 96
image_height, image_width = IMG_SIZE, IMG_SIZE
no_of_color_channels = 3
root_dir = os.getcwd()
db_dir = f"{root_dir}/ensemble_db"
no_of_classes = len(os.listdir(db_dir))
print(root_dir)

total_no_of_images = len(
    list(glob.glob(f"{db_dir}/[0-9]*/*.*", recursive=True)))
print(total_no_of_images)

count = 0
# test_size = 2 # from each class
test_size = int((total_no_of_images / 56) * 10 / 100)  # from each class
train_size = total_no_of_images - no_of_classes * test_size
data = np.empty((train_size, image_height, image_width, no_of_color_channels))
labels = np.empty(train_size, dtype=int)

test_count = 0
test_data = np.empty(
    (no_of_classes * test_size, image_height, image_width, no_of_color_channels)
)
test_labels = np.empty(no_of_classes * test_size, dtype=int)

print(train_size)
print(no_of_classes * test_size)

for i in range(no_of_classes):
    class_dir = root_dir + "/" + "mmdb_400" + "/" + str(i)
    class_images = glob.glob(class_dir + "/*.*")
    np.random.shuffle(class_images)
    size = len(class_images)
    print(i, end=" ") if i == 0 or i % 10 != 0 else print(i)
    for j in range(size):
        image_path = class_images[j]
        image = cv2.imread(image_path)
        # image = cv2.resize(image, (32, 32))
        if j < size - test_size:
            data[count] = image
            labels[count] = i
            count += 1
        else:
            test_data[test_count] = image
            test_labels[test_count] = i
            test_count += 1
print("")

c:\Users\D\Documents\00Final_Year_Project\training_and_testing
22400
20160
2240
0 

ValueError: could not broadcast input array from shape (13,8,3) into shape (96,96,3)

In [ ]:
train_data_96, train_labels = shuffle(data, labels)
train_data_71 = []
train_data_32 = []

test_data_96 = test_data
test_data_71 = []
test_data_32 = []

for image in train_data_96:
    train_data_71.append(cv2.resize(image, (71, 71)))
    train_data_32.append(cv2.resize(image, (32, 32)))
train_data_96 = np.asarray(train_data_96)
train_data_71 = np.asarray(train_data_71)
train_data_32 = np.asarray(train_data_32)

for image in test_data_96:
    test_data_71.append(cv2.resize(image, (71, 71)))
    test_data_32.append(cv2.resize(image, (32, 32)))
test_data_96 = np.asarray(test_data_96)
test_data_71 = np.asarray(test_data_71)
test_data_32 = np.asarray(test_data_32)


In [ ]:
train_data = {
    "vgg16": vgg16.preprocess_input(train_data_32),
    "vgg19": vgg19.preprocess_input(train_data_32),
    "resnet": resnet_v2.preprocess_input(train_data_32),
    "xception": xception.preprocess_input(train_data_71),
    "mobilenet": mobilenet_v2.preprocess_input(train_data_96),
}

test_data = {
    "vgg16": vgg16.preprocess_input(test_data_32),
    "vgg19": vgg19.preprocess_input(test_data_32),
    "resnet": resnet_v2.preprocess_input(test_data_32),
    "xception": xception.preprocess_input(test_data_71),
    "mobilenet": mobilenet_v2.preprocess_input(test_data_96),
}

In [ ]:
# # Preprocess training and testing data

# print(data.min(), data.max())
# print(test_data.min(), test_data.max())

# # for i in range(len(data)):
# #   data[i] = preprocess_input(data[i])
# # for i in range(len(test_data)):
# #   test_data[i] = preprocess_input(test_data[i])

# data = vgg16.preprocess_input(data)
# test_data = vgg16.preprocess_input(test_data)

# data_2 = xception.preprocess_input(data_2)
# test_data_2 = xception.preprocess_input(test_data_2)

# print(data.min(), data.max())
# print(test_data.min(), test_data.max())
# print(data_2.min(), data_2.max())
# print(test_data_2.min(), test_data_2.max())


0.0 255.0
0.0 255.0
-123.68 151.061
-123.68 151.061
-1.0 1.0
-1.0 1.0


In [ ]:
def get_appropriate_data(model_name, data):
    if "vgg16" in model_name:
        x = data["vgg16"]
    elif "vgg19" in model_name:
        x = data["vgg19"]
    elif "resnet" in model_name:
        x = data["resnet"]
    elif "mobilenet" in model_name:
        x = data["mobilenet"]
    elif "xception" in model_name:
        x = data["xception"]
    return x

In [ ]:
# evaluate standalone models on test dataset
for i, model in enumerate(members):
	# testy_enc = to_categorical(test_labels)
	test_y = test_labels
	model_name = model.layers[0].name
	test_x = get_appropriate_data(model_name, test_data)
	_, acc = model.evaluate(test_x, test_y, verbose='auto')
	print('Model Accuracy: %.3f' % acc)

Model Accuracy: 0.984
Model Accuracy: 0.983
Model Accuracy: 0.974


In [ ]:
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
    stackX = None
    for i, model in enumerate(members):
        # make prediction
        # yhat = model.predict(inputX[1] if i == 2 else inputX[0], verbose=0)
        inputX = get_appropriate_data(model.layers[0].name, inputX)
        yhat = model.predict(inputX)
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = np.dstack((stackX, yhat))
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape(
        (stackX.shape[0], stackX.shape[1] * stackX.shape[2]))
    return stackX


In [ ]:
# # fit a model based on the outputs from the ensemble members
# def fit_stacked_model(members, inputX, inputy):
# 	# create dataset using ensemble
# 	print("e")
# 	stackedX = stacked_dataset(members, inputX)
# 	# fit standalone model
# 	model = LogisticRegression()
# 	# Callbacks: EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


# 	# es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)
# 	earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0,
# 							patience=10, verbose=1, restore_best_weights=True)
# 	checkpoint = ModelCheckpoint(
# 		'mobilenetv2_mmdb_400.h5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)
# 	# rlronp = ReduceLROnPlateau( monitor="val_accuracy", factor=0.005,patience=1, verbose=1)
# 	rlronp = ReduceLROnPlateau(
# 		monitor='val_loss', factor=0.5, patience=5, verbose=1,  min_lr=0.00001)

# 	callbacks = [earlystop, checkpoint, rlronp]
# 	print("fitting...")
# 	model.fit(stackedX, inputy, epochs=200,
#            shuffle=True,
#            verbose='auto',
#            callbacks=callbacks,
#            validation_split=0.1)
# 	# model.fit(stackedX, inputy)
# 	return model


# # make a prediction with the stacked model
# def stacked_prediction(members, model, inputX):
# 	# create dataset using ensemble
# 	stackedX = stacked_dataset(members, inputX)
# 	# make a prediction
# 	yhat = model.predict(stackedX)
# 	return yhat

In [ ]:
# # fit stacked model using the ensemble
# model = fit_stacked_model(members, train_data, train_labels)
# # evaluate model on test set
# yhat = stacked_prediction(members, model, test_data)
# acc = accuracy_score(test_labels, yhat)
# print('Stacked Test Accuracy: %.3f' % acc)

In [ ]:
stackedX = stacked_dataset(members, train_data)
test_stackedX = stacked_dataset(members, test_data)

69/69 [==============================] - 39s 559ms/step


In [ ]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(stackedX, inputy):
    # fit standalone model
    model = LogisticRegression()
    model.fit(stackedX, inputy)
    return model


# make a prediction with the stacked model
def stacked_prediction(model, test_stackedX):
    # make a prediction
    yhat = model.predict(test_stackedX)
    return yhat


In [ ]:
# fit stacked model using the ensemble
model = fit_stacked_model(stackedX, train_labels)
# evaluate model on test set
yhat = stacked_prediction(model, test_stackedX)
acc = accuracy_score(test_labels, yhat)
print("Stacked Test Accuracy: %.3f" % acc)


Stacked Test Accuracy: 0.995
